In [2]:
!pip install tqdm
!conda install -y Pillow

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re

from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, LSTM, Embedding, Reshape
from keras.models import load_model, model_from_json

from sklearn.model_selection import train_test_split



'conda' is not recognized as an internal or external command,
operable program or batch file.
c:\users\arpit\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
from keras.engine.topology import Layer
import keras.backend as K
from keras import initializers
import numpy as np


In [3]:
df = pd.read_csv('fake_or_real_news.csv')

In [4]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
df.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [7]:
df.title = df.title.str.lower()
df.text = df.text.str.lower()

In [24]:
df.title = df.title.str.replace(r'http[\w:/\.]+','<URL>') # remove urls
df.text = df.text.str.replace(r'http[\w:/\.]+','<URL>') # remove urls
df.title = df.title.str.replace(r'[^\.\w\s]','') #remove everything but characters and punctuation
df.text = df.text.str.replace(r'[^\.\w\s]','') #remove everything but characters and punctuation
df.title = df.title.str.replace(r'\.\.+','.') #replace multple periods with a single one
df.text = df.text.str.replace(r'\.\.+','.') #replace multple periods with a single one

with open('./glove.6B.50d.txt','rb') as f:
    lines = f.readlines()
    
glove_weights = np.zeros((len(lines), 50))
words = []
for i, line in enumerate(lines):
    word_weights = line.split()
    words.append(word_weights[0])
    weight = word_weights[1:]
    glove_weights[i] = np.array([float(w) for w in weight])
word_vocab = [w.decode("utf-8") for w in words]

word2glove = dict(zip(word_vocab, glove_weights))
df.title = df.title.str.replace(r'\.',' . ') #replace periods with a single one
df.text = df.text.str.replace(r'\.',' . ') #replace multple periods with a single one
df.title = df.title.str.replace(r'\s\s+',' ') #replace multple white space with a single one
df.text = df.text.str.replace(r'\s\s+',' ') #replace multple white space with a single one
df.title = df.title.str.strip() 
df.text = df.text.str.strip() 

In [9]:
df.head()

,title,text,label
0,you can smell hillarys fear,daniel greenfield a shillman journalism fellow...,FAKE
1,watch the exact moment paul ryan committed pol...,google pinterest digg linkedin reddit stumbleu...,FAKE
2,kerry to go to paris in gesture of sympathy,u . s . secretary of state john f . kerry said...,REAL
3,bernie supporters on twitter erupt in anger ag...,kaydee king kaydeeking november 9 2016 the les...,FAKE
4,the battle of new york why this primary matters,its primary day in new york and frontrunners h...,REAL


In [10]:
all_text = ' '.join(df.text.values)
words = all_text.split()

In [11]:
words

['daniel',
 'greenfield',
 'a',
 'shillman',
 'journalism',
 'fellow',
 'at',
 'the',
 'freedom',
 'center',
 'is',
 'a',
 'new',
 'york',
 'writer',
 'focusing',
 'on',
 'radical',
 'islam',
 '.',
 'in',
 'the',
 'final',
 'stretch',
 'of',
 'the',
 'election',
 'hillary',
 'rodham',
 'clinton',
 'has',
 'gone',
 'to',
 'war',
 'with',
 'the',
 'fbi',
 '.',
 'the',
 'word',
 'unprecedented',
 'has',
 'been',
 'thrown',
 'around',
 'so',
 'often',
 'this',
 'election',
 'that',
 'it',
 'ought',
 'to',
 'be',
 'retired',
 '.',
 'but',
 'its',
 'still',
 'unprecedented',
 'for',
 'the',
 'nominee',
 'of',
 'a',
 'major',
 'political',
 'party',
 'to',
 'go',
 'war',
 'with',
 'the',
 'fbi',
 '.',
 'but',
 'thats',
 'exactly',
 'what',
 'hillary',
 'and',
 'her',
 'people',
 'have',
 'done',
 '.',
 'coma',
 'patients',
 'just',
 'waking',
 'up',
 'now',
 'and',
 'watching',
 'an',
 'hour',
 'of',
 'cnn',
 'from',
 'their',
 'hospital',
 'beds',
 'would',
 'assume',
 'that',
 'fbi',
 'dire

In [14]:
from collections import Counter
u_words = Counter(words).most_common()
u_words

[('the', 290455),
 ('.', 248315),
 ('to', 139265),
 ('of', 129571),
 ('and', 119263),
 ('a', 108266),
 ('in', 98052),
 ('that', 69337),
 ('is', 54344),
 ('for', 46302),
 ('on', 40601),
 ('as', 32517),
 ('it', 31837),
 ('with', 30837),
 ('he', 29400),
 ('was', 27088),
 ('are', 25641),
 ('this', 25048),
 ('be', 24586),
 ('by', 24388),
 ('have', 24228),
 ('not', 22695),
 ('his', 22366),
 ('has', 22357),
 ('at', 21608),
 ('said', 21190),
 ('but', 21151),
 ('from', 20656),
 ('they', 18435),
 ('trump', 18433),
 ('i', 18412),
 ('an', 18033),
 ('who', 17324),
 ('we', 16506),
 ('will', 15345),
 ('you', 14635),
 ('its', 14564),
 ('clinton', 14429),
 ('or', 14354),
 ('about', 14252),
 ('their', 13977),
 ('more', 12943),
 ('would', 12666),
 ('were', 11974),
 ('if', 11544),
 ('been', 11419),
 ('all', 11416),
 ('people', 11374),
 ('her', 11314),
 ('one', 11219),
 ('what', 10788),
 ('she', 10516),
 ('had', 10223),
 ('which', 9779),
 ('when', 9543),
 ('out', 9434),
 ('new', 9268),
 ('so', 9096),
 ('th

In [19]:
u_words_counter = u_words
u_words_frequent = [word[0] for word in u_words if word[1]>20] # we will only consider words that have been used more than 5 times
u_words_counter

[('the', 290455),
 ('.', 248315),
 ('to', 139265),
 ('of', 129571),
 ('and', 119263),
 ('a', 108266),
 ('in', 98052),
 ('that', 69337),
 ('is', 54344),
 ('for', 46302),
 ('on', 40601),
 ('as', 32517),
 ('it', 31837),
 ('with', 30837),
 ('he', 29400),
 ('was', 27088),
 ('are', 25641),
 ('this', 25048),
 ('be', 24586),
 ('by', 24388),
 ('have', 24228),
 ('not', 22695),
 ('his', 22366),
 ('has', 22357),
 ('at', 21608),
 ('said', 21190),
 ('but', 21151),
 ('from', 20656),
 ('they', 18435),
 ('trump', 18433),
 ('i', 18412),
 ('an', 18033),
 ('who', 17324),
 ('we', 16506),
 ('will', 15345),
 ('you', 14635),
 ('its', 14564),
 ('clinton', 14429),
 ('or', 14354),
 ('about', 14252),
 ('their', 13977),
 ('more', 12943),
 ('would', 12666),
 ('were', 11974),
 ('if', 11544),
 ('been', 11419),
 ('all', 11416),
 ('people', 11374),
 ('her', 11314),
 ('one', 11219),
 ('what', 10788),
 ('she', 10516),
 ('had', 10223),
 ('which', 9779),
 ('when', 9543),
 ('out', 9434),
 ('new', 9268),
 ('so', 9096),
 ('th

In [16]:
u_words_frequent

['the',
 '.',
 'to',
 'of',
 'and',
 'a',
 'in',
 'that',
 'is',
 'for',
 'on',
 'as',
 'it',
 'with',
 'he',
 'was',
 'are',
 'this',
 'be',
 'by',
 'have',
 'not',
 'his',
 'has',
 'at',
 'said',
 'but',
 'from',
 'they',
 'trump',
 'i',
 'an',
 'who',
 'we',
 'will',
 'you',
 'its',
 'clinton',
 'or',
 'about',
 'their',
 'more',
 'would',
 'were',
 'if',
 'been',
 'all',
 'people',
 'her',
 'one',
 'what',
 'she',
 'had',
 'which',
 'when',
 'out',
 'new',
 'so',
 'there',
 'up',
 'no',
 'than',
 'can',
 'our',
 'state',
 'also',
 'president',
 'other',
 'us',
 'campaign',
 'after',
 'just',
 'some',
 'do',
 'over',
 'like',
 'them',
 'into',
 'states',
 'hillary',
 'obama',
 'could',
 'now',
 'time',
 'even',
 'only',
 'because',
 'how',
 'him',
 'most',
 'many',
 'republican',
 's',
 'u',
 'those',
 'first',
 'these',
 'two',
 'any',
 'against',
 'election',
 'years',
 'political',
 'government',
 'get',
 'american',
 'party',
 'while',
 'house',
 'going',
 'your',
 'world',
 'be

In [20]:
u_words_total = [k for k,v in u_words_counter]
u_words_total

['the',
 '.',
 'to',
 'of',
 'and',
 'a',
 'in',
 'that',
 'is',
 'for',
 'on',
 'as',
 'it',
 'with',
 'he',
 'was',
 'are',
 'this',
 'be',
 'by',
 'have',
 'not',
 'his',
 'has',
 'at',
 'said',
 'but',
 'from',
 'they',
 'trump',
 'i',
 'an',
 'who',
 'we',
 'will',
 'you',
 'its',
 'clinton',
 'or',
 'about',
 'their',
 'more',
 'would',
 'were',
 'if',
 'been',
 'all',
 'people',
 'her',
 'one',
 'what',
 'she',
 'had',
 'which',
 'when',
 'out',
 'new',
 'so',
 'there',
 'up',
 'no',
 'than',
 'can',
 'our',
 'state',
 'also',
 'president',
 'other',
 'us',
 'campaign',
 'after',
 'just',
 'some',
 'do',
 'over',
 'like',
 'them',
 'into',
 'states',
 'hillary',
 'obama',
 'could',
 'now',
 'time',
 'even',
 'only',
 'because',
 'how',
 'him',
 'most',
 'many',
 'republican',
 's',
 'u',
 'those',
 'first',
 'these',
 'two',
 'any',
 'against',
 'election',
 'years',
 'political',
 'government',
 'get',
 'american',
 'party',
 'while',
 'house',
 'going',
 'your',
 'world',
 'be

In [25]:
word_vocab = dict(zip(word_vocab, range(len(word_vocab))))

In [26]:
# Import `fake_or_real_news.csv` 
df = pd.read_csv('fake_or_real_news.csv')
    
# Inspect shape of `df` 
df.shape

# Print first lines of `df` 
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [27]:
# Set index 
df = df.set_index("Unnamed: 0")

# Print first lines of `df` 
df.head()

,title,text,label
Unnamed: 0,,,
8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr 27 19:11:57 2018

@author: arpit
"""

# Natural Language Processing

# Importing the libraries
# Natural Language Processing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('fake_or_real_news.csv')
dataset.drop('Unnamed: 0', axis=1, inplace=True)
dataset.title = dataset.title.str.lower()
dataset.text = dataset.text.str.lower()
dataset.title = dataset.title.str.replace(r'http[\w:/\.]+','<URL>') # remove urls
dataset.text = dataset.text.str.replace(r'http[\w:/\.]+','<URL>') # remove urls
dataset.title = dataset.title.str.replace(r'[^\.\w\s]','') #remove everything but characters and punctuation
dataset.text = dataset.text.str.replace(r'[^\.\w\s]','') #remove everything but characters and punctuation
dataset.title = dataset.title.str.replace(r'\.\.+','.') #replace multple periods with a single one
dataset.text = dataset.text.str.replace(r'\.\.+','.') #replace multple periods with a single one
dataset.title = dataset.title.str.replace(r'\.',' . ') #replace periods with a single one
dataset.text = dataset.text.str.replace(r'\.',' . ') #replace multple periods with a single one
dataset.title = dataset.title.str.replace(r'\s\s+',' ') #replace multple white space with a single one
dataset.text = dataset.text.str.replace(r'\s\s+',' ') #replace multple white space with a single one

# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
y = dataset.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(dataset['text'], y, test_size = 0.20, random_state = 0)
#print(review)
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')
count_train = cv.fit_transform(X_train.values)
count_test=count_train = cv.transform(X_test.values)


print(type(count_test))

# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(count_train, y_test)

# Predicting the Test set results
y_pred = classifier.predict(count_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)

<class 'scipy.sparse.csr.csr_matrix'>


0.9558011049723757

In [3]:
cm

array([[562,  53],
       [  3, 649]], dtype=int64)